### Import dependencies :

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict,ShuffleSplit,GridSearchCV,KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error , mean_absolute_error
from math import sqrt
import joblib
import json
import datetime

2025-06-10 22:49:39.965037: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 22:49:40.243300: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 22:49:40.503996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749588580.752465   32530 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749588580.823510   32530 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749588581.297039   32530 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
import pandas as pd
import numpy as np
df_no_outliers = pd.read_csv('/home/fayari/Stage/data_exploration_2/no_outliers.csv')
new = pd.read_csv('/home/fayari/Stage/modeling_BOA/add_training_v2.csv')

# Séparer les features et la cible
y_add = new['alpha']
X_add = new.drop(columns=['alpha'])

# Paramètres
n_copies = 650
X_augmented = []
y_augmented = []

# Générer des copies bruitées
for _ in range(n_copies):
    noise = np.random.normal(loc=0.0, scale=0.01, size=X_add.shape)
    X_aug = X_add.values + noise
    X_augmented.append(X_aug)
    y_augmented.append(y_add.values)

# Empiler les données
X_augmented = np.vstack(X_augmented)
y_augmented = np.hstack(y_augmented)

# Reformer un DataFrame avec les colonnes originales
df_new_augmented = pd.DataFrame(
    np.hstack([X_augmented, y_augmented.reshape(-1, 1)]),
    columns=new.columns
)

# Fusionner avec df_no_outliers
df_augmented = pd.concat([df_no_outliers, df_new_augmented], ignore_index=True)

In [3]:
df_augmented

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha
0,0.294500,0.068532,0.953130,0.940937,1.222476,0.812762,0.764758,32.000000,225.000000,0.098065,0.916053,0.848048,707.913889,3185.612500,0.848282,0.848100,1.149748,1.053646,19.107875,0.766006
1,0.302615,0.120446,0.953829,0.941623,1.195295,0.809747,0.762476,32.000000,225.000000,0.779041,0.915792,0.848048,707.913889,3185.612500,0.848282,0.848100,1.149748,1.053646,49.592110,0.415972
2,0.310783,0.183238,0.954528,0.942295,1.168659,0.806488,0.759950,32.000000,225.000000,0.370995,0.915532,0.848048,707.913889,3185.612500,0.848282,0.848100,1.149748,1.053646,72.148612,0.543631
3,0.318875,0.243437,0.955227,0.942991,1.142956,0.803487,0.757681,32.000000,225.000000,0.241212,0.915271,0.848048,707.913889,3185.612500,0.848282,0.848100,1.149748,1.053646,99.018854,0.566964
4,0.326900,0.308727,0.955787,0.943509,1.118101,0.800492,0.755271,32.000000,225.000000,0.033981,0.915010,0.848048,707.913889,3185.612500,0.848282,0.848100,1.149748,1.053646,124.868584,0.629959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886745,1.001150,0.783395,0.992109,0.847058,-0.021818,0.026489,0.028190,15.006230,59.021910,0.790729,0.889644,0.961922,707.901412,3185.521670,0.968840,0.969476,1.040402,0.996376,776.673618,-0.218157
1886746,0.986933,0.872268,1.002022,0.831011,0.002297,0.054127,0.036699,14.984698,59.014714,0.795509,0.894563,0.979416,707.892397,3185.526816,0.961248,0.981356,1.008069,0.999625,875.099431,-0.051697
1886747,1.000374,0.854945,0.984600,0.817928,0.004184,0.023073,0.032757,15.000706,58.991118,0.800755,0.885347,0.976809,707.907070,3185.525602,0.949255,0.978790,1.026743,0.985003,843.468220,-0.154843
1886748,0.991381,0.831727,0.998383,0.830063,0.005587,0.025237,0.020427,14.988263,59.011692,0.796450,0.892745,0.962113,707.888462,3185.511236,0.978011,0.984324,1.033827,1.018744,824.402839,-0.200728


### Load data :

In [7]:
df=pd.read_csv('preprocessed_data_.csv')
df.head()

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4,BOA_RT,alpha
0,0.938938,0.830099,-0.143116,0.205274,-0.748201,-0.102195,-0.148214,1.494961,-0.196407,-0.449844,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,429.332315,0.133511
1,0.072507,-1.551574,1.189851,0.880461,-0.191308,-0.932700,-0.652588,1.032032,1.082365,-0.470705,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,194.210750,0.363279
2,-2.603383,0.858439,-0.191497,0.141423,2.954771,-0.107965,-0.172161,-1.032376,-0.241683,0.598065,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,648.404343,0.280041
3,0.181490,0.807331,1.187661,-1.173034,-0.267672,-1.155707,-0.908316,0.025438,-1.062994,0.958397,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,885.138727,0.125099
4,-0.956225,0.171426,-0.715124,0.448112,0.653581,0.622586,0.469534,-0.757894,-0.363971,0.962627,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,602.697524,0.320636


In [20]:
X=df.drop(columns=['alpha'])
y=df['alpha']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
BOA_train=X_train['BOA_RT']
BOA_test=X_test['BOA_RT']

In [25]:
X_train

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_0,mprime_a_bin_1,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4
80661,-0.069120,0.880779,0.640769,-0.173369,-0.088895,-0.790171,-0.665889,1.520224,0.279489,1.004118,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
248726,-1.330278,-2.773945,-0.221436,-0.573381,1.036162,-0.425127,-0.545665,-1.069706,1.448924,0.855865,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
247518,-0.447697,-1.204405,1.493352,-0.202837,0.204542,-1.268209,-0.942720,0.247516,0.436509,-0.688671,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
303607,-1.111384,0.874273,0.948692,-1.774713,0.806064,-1.098430,-0.918408,0.994639,-0.978477,-0.442122,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
305103,-3.228436,0.674241,-1.167519,0.118035,4.781522,0.640134,0.312535,-0.022583,-0.726047,1.270408,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.752914,0.744033,0.041159,-1.205458,-0.637513,-0.706761,-0.774369,-0.775916,0.129560,0.250184,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
259178,0.930569,0.612489,-1.441607,0.123058,-0.743313,0.876728,0.471052,-1.627810,-0.521755,-1.252927,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
131932,0.923727,0.498173,-0.628089,-1.309832,-0.739311,-0.445585,-0.719076,-0.930604,0.217773,-0.864780,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
146867,0.379052,0.323402,0.139977,0.378624,-0.401067,-0.210751,-0.172633,1.146767,0.084081,1.734614,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [26]:
y.describe()

count    362503.000000
mean          0.288921
std           0.204526
min          -0.284378
25%           0.166093
50%           0.312022
75%           0.430461
max           0.853075
Name: alpha, dtype: float64

### Load Model :

In [30]:
# Load best parameters from JSON file
with open("best_params.json", "r") as f:
    params = json.load(f)

# Build the model
model = Sequential()

# Layer 1
model.add(Dense(params["units1"], activation=params["activation"], input_shape=(X_train.shape[1],)))
model.add(Dropout(params["dropout1"]))

# Layer 2
model.add(Dense(params["units2"], activation=params["activation"]))
model.add(Dropout(params["dropout2"]))

# Layer 3 (if applicable)
if params["n_layers"] >= 3:
    model.add(Dense(params["units3"], activation=params["activation"]))
    model.add(Dropout(params["dropout3"]))

# Layer 4 (if applicable)
if params["n_layers"] == 4:
    model.add(Dense(params["units4"], activation=params["activation"]))
    model.add(Dropout(params["dropout4"]))

# Output layer
model.add(Dense(1))

# Compile model
optimizer = Adam(learning_rate=params["lr"])
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])


In [31]:
# 1. Early stopping
early_stop = EarlyStopping(
    monitor='val_mae',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# 2. Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_mae',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# 3. Model checkpoint (save best model)
checkpoint_cb = ModelCheckpoint(
    filepath='deep_model1.keras',
    monitor='val_mae',
    save_best_only=True,
    verbose=1
)

# 4. CSV logger (log training history)
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
csv_logger = CSVLogger(f'training_log_{timestamp}.csv')

# Train phase :

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=params["batch_size"],
    callbacks=[early_stop, reduce_lr, checkpoint_cb, csv_logger],
    verbose=1
)


Epoch 1/200
3388/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0286 - mae: 0.1198
Epoch 1: val_mae improved from inf to 0.04726, saving model to deep_model1.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 0.0285 - mae: 0.1197 - val_loss: 0.0045 - val_mae: 0.0473 - learning_rate: 3.4066e-04
Epoch 2/200
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0063 - mae: 0.0580
Epoch 2: val_mae improved from 0.04726 to 0.03454, saving model to deep_model1.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0063 - mae: 0.0580 - val_loss: 0.0025 - val_mae: 0.0345 - learning_rate: 3.4066e-04
Epoch 3/200
3392/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0034 - mae: 0.0418
Epoch 3: val_mae improved from 0.03454 to 0.02880, saving model to deep_model1.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 0.0034 - mae: 0.0418 - val_loss: 0.0017 - val_mae: 0.0288 - learning_rate: 3.4066e-04
Epoch 4/200
3391/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0023 - mae: 0.0337

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362503 entries, 0 to 362502
Data columns (total 30 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Tg_scat         362503 non-null  float64
 1   Tg_abs          362503 non-null  float64
 2   Ta_abs          362503 non-null  float64
 3   SSA             362503 non-null  float64
 4   GOD             362503 non-null  float64
 5   AOD             362503 non-null  float64
 6   AODS            362503 non-null  float64
 7   SZA             362503 non-null  float64
 8   Z               362503 non-null  float64
 9   R_scence        362503 non-null  float64
 10  g1              362503 non-null  float64
 11  Cos(SZA)        362503 non-null  float64
 12  mu_g            362503 non-null  float64
 13  mu_a            362503 non-null  float64
 14  muprime_g       362503 non-null  float64
 15  muprime_a       362503 non-null  float64
 16  mprime_g        362503 non-null  float64
 17  mprime_a  

In [11]:
model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 9,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 300,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271877 entries, 80661 to 121958
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Tg_scat         271877 non-null  float64
 1   Tg_abs          271877 non-null  float64
 2   Ta_abs          271877 non-null  float64
 3   SSA             271877 non-null  float64
 4   GOD             271877 non-null  float64
 5   AOD             271877 non-null  float64
 6   AODS            271877 non-null  float64
 7   SZA             271877 non-null  float64
 8   Z               271877 non-null  float64
 9   R_scence        271877 non-null  float64
 10  g1              271877 non-null  float64
 11  Cos(SZA)        271877 non-null  float64
 12  mu_g            271877 non-null  float64
 13  mu_a            271877 non-null  float64
 14  muprime_g       271877 non-null  float64
 15  muprime_a       271877 non-null  float64
 16  mprime_g        271877 non-null  float64
 17  mprime_a   

In [6]:
y_train

80661     0.723401
248726    0.415715
247518    0.391787
303607    0.244650
305103    0.475617
            ...   
119879    0.474890
259178    0.342325
131932   -0.104851
146867    0.504175
121958    0.103362
Name: alpha, Length: 271877, dtype: float64

In [8]:
model=joblib.load('model.pkl')
model.predict(X_test)

array([0.8401935, 1.4607283, 1.2887543, ..., 1.2130278, 1.1415201,
       1.2756097], dtype=float32)

In [14]:
X_np = X_train.to_numpy()
y_np = y_train.to_numpy().ravel()

In [5]:
from tensorflow.keras.models import load_model

model = load_model('deep_model.h5', compile=False)


In [10]:
y_pred=model.predict(X_test)

2833/2833 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [11]:
y_pred

array([[-0.21968824],
       [ 0.3766121 ],
       [ 0.22941366],
       ...,
       [ 0.1528129 ],
       [ 0.14012328],
       [ 0.23602526]], dtype=float32)

In [12]:
y_test

159680   -0.224256
347342    0.376680
307069    0.238902
126659    0.471005
356896    0.128247
            ...   
326002    0.194530
190999   -0.142424
27703     0.149952
76624     0.143431
189606    0.229081
Name: alpha, Length: 90626, dtype: float64

In [13]:
mean_absolute_error(y_pred,y_test)

0.004759879721172527

In [16]:
X_test

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_0,mprime_a_bin_1,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4
159680,0.968688,0.690381,-1.610247,-0.842480,-0.765509,0.221345,-0.349359,-1.019351,2.036857,1.369018,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
347342,-1.203300,0.908147,0.427997,-0.205544,0.900421,-0.673723,-0.603632,0.725515,-0.216203,-0.280011,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
307069,-0.052629,-0.103307,1.356170,-2.028452,-0.101012,-1.229507,-0.940427,1.285479,-1.116649,0.798446,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
126659,0.122052,-1.145261,-1.486535,-0.352838,-0.226280,0.457653,-0.026960,0.770059,2.211000,-0.862565,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
356896,-0.314747,0.806773,-0.039354,-1.604842,0.097997,-0.740746,-0.841591,-1.634395,-0.617101,1.323553,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.570257,0.210242,0.225099,-0.963130,-0.524454,-0.741572,-0.750298,-1.102005,0.164011,0.682795,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
190999,-0.122803,-1.748566,0.261369,1.172320,-0.049088,0.707174,0.919019,1.778795,0.595663,1.627898,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
27703,0.718968,0.367986,0.122163,0.103691,-0.616840,-0.358504,-0.347800,-0.751896,-1.091658,0.908998,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
76624,0.015910,0.770747,1.477093,-1.292452,-0.150825,-1.263014,-0.941725,-0.509560,-0.099483,0.205600,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [17]:
cols_to_scale_inv = X_test.drop(columns=['mprime_a_bin_0', 'mprime_a_bin_1',
       'mprime_a_bin_2', 'mprime_a_bin_3', 'mprime_a_bin_4', 'mprime_g_bin_0',
       'mprime_g_bin_1', 'mprime_g_bin_2', 'mprime_g_bin_3', 'mprime_g_bin_4'
       ]).columns
cols_to_encode_inv = ['mprime_a_bin_0', 'mprime_a_bin_1',
       'mprime_a_bin_2', 'mprime_a_bin_3', 'mprime_a_bin_4', 'mprime_g_bin_0',
       'mprime_g_bin_1', 'mprime_g_bin_2', 'mprime_g_bin_3', 'mprime_g_bin_4',
       ]

In [18]:
scaler=joblib.load('scaler.pkl')
encoder=joblib.load('encoder.pkl')
X_scaled_part = X_test[cols_to_scale_inv].values
X_encoded_part = X_test[cols_to_encode_inv].values
# 2. Inverser la standardisation
X_original_scaled = scaler.inverse_transform(X_scaled_part)

# 3. Inverser l'encodage
X_original_categoricals = encoder.inverse_transform(X_encoded_part)

In [19]:
df_reconstructed = pd.DataFrame(
    X_original_scaled,
    columns=['Tg_scat', 'Tg_abs', 'Ta_abs', 'SSA', 'GOD', 'AOD', 'AODS', 'SZA', 'Z',
       'R_scence', 'g1', 'Cos(SZA)', 'mu_g', 'mu_a', 'muprime_g', 'muprime_a',
       'mprime_g', 'mprime_a'],
    index=X_test.index
)
df_categoricals = pd.DataFrame(
    X_original_categoricals,
    columns=['mprime_a_bin', 'mprime_g_bin'],
    index=X_test.index
)
X_test_real = pd.concat([df_reconstructed, df_categoricals], axis=1)

In [21]:
X_test_real['alpha_pred']=y_pred
X_test_real['alpha']=y_test
X_test_real['BOA_RT']=BOA_test

In [22]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,alpha_pred,alpha,BOA_RT
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,3187.429121,0.951776,0.951721,0.684362,0.152645,0,0,-0.219688,-0.224256,914.444755
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,3186.056472,0.471695,0.470791,1.873413,1.217586,0,0,0.376612,0.376680,594.575497
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,3185.507886,0.243492,0.241375,4.099723,4.110381,0,0,0.229414,0.238902,284.591943
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,3187.535216,0.454355,0.453397,1.400187,0.288163,0,0,0.468541,0.471005,373.718733
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,3185.812230,0.999159,0.999158,0.933753,0.732430,0,0,0.126366,0.128247,832.137702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,3186.288112,0.962308,0.962266,0.872219,0.472533,0,0,0.194265,0.194530,801.588284
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,3186.551091,0.042399,0.030856,18.672395,11.328534,0,2,-0.150260,-0.142424,32.089238
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,3185.523112,0.909141,0.909037,1.094304,1.074933,0,0,0.152813,0.149952,735.627029
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,3186.127582,0.859623,0.859456,1.011868,0.621184,0,0,0.140123,0.143431,924.379368


In [23]:
def calculate_BOA_IA(Tg_abs,Mg,Ta_abs,Ma,alpha,delta_g_scat,AODS):
    numerator= 1000*(Tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*AODS)*Ma
    if denominator != 0 :
        BOA_ia= numerator / denominator
        return BOA_ia
    else :
        return 999

In [25]:
X_test_real['BOA_AI'] = X_test_real.apply(lambda row: calculate_BOA_IA(row['Tg_abs'], row['mprime_g'],row['Ta_abs'],row['mprime_a'],row['alpha_pred'],row['GOD'],row['AODS']), axis=1)

In [26]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,alpha_pred,alpha,BOA_RT,BOA_AI
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951776,0.951721,0.684362,0.152645,0,0,-0.219688,-0.224256,914.444755,914.378481
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.471695,0.470791,1.873413,1.217586,0,0,0.376612,0.376680,594.575497,594.608170
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.243492,0.241375,4.099723,4.110381,0,0,0.229414,0.238902,284.591943,286.792481
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.454355,0.453397,1.400187,0.288163,0,0,0.468541,0.471005,373.718733,373.965582
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999159,0.999158,0.933753,0.732430,0,0,0.126366,0.128247,832.137702,832.591657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962308,0.962266,0.872219,0.472533,0,0,0.194265,0.194530,801.588284,801.607407
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.042399,0.030856,18.672395,11.328534,0,2,-0.150260,-0.142424,32.089238,-0.002811
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909141,0.909037,1.094304,1.074933,0,0,0.152813,0.149952,735.627029,735.353831
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859623,0.859456,1.011868,0.621184,0,0,0.140123,0.143431,924.379368,925.058904


In [ ]:
def calculate_BOA_TOTAL(SZA,Z,alpha,l,E_TOA):   
    def muprime(z,h,µ):
        RAYON_TERRESTRE=6371
        eta = (RAYON_TERRESTRE*1000 + z) / h
        root = (eta*µ)**2  + 2 * eta + 1
        sum = (root)**0.5 - eta * µ
        if sum > 0 :
            return 1/sum
        return 1 
    BOA=[]
    for i in range(len(l)) :
        tg_scat, tg_abs, optical_depth, albedo, a, g1 = l[i]
        Ha=2000
        Hg=9000
        angle_rad = math.radians(SZA)
        µ=math.cos(angle_rad)  
        Y_a=muprime(Z,Ha,µ)
        Y_g=muprime(Z,Hg,µ)
        Ma=math.exp(-Z/Ha)/Y_a
        Mg=math.exp(-Z/Hg)/Y_g
        delta_a_scat=optical_depth*albedo
        delta_g_scat=-math.log(tg_scat)
        Ta_abs=math.exp(-optical_depth*(1-albedo))
        numerator= E_TOA[i]*(tg_abs**Mg)*(Ta_abs**Ma)
        denominator=1+alpha[i]*delta_g_scat*Mg+(alpha[i]*(1/3)*delta_a_scat)*Ma
        BOA_i= numerator / denominator
        BOA.append(BOA_i)
    return BOA